In [1]:
# basic setup
import json
import os
import random
from PIL import Image
from tqdm.notebook import tqdm
from glob import glob
from collections import defaultdict

In [10]:
# 샘플 데이터 들어있는 폴더 경로(이름을 아래처럼 바꿔주었습니다)
# 내부에 백향과, 백향과 json, .... 등의 폴더로 구성
base_dir='./data_sample/'

In [90]:
def convert_to_COCO(base_dir,save_dir='./'):

    json_folder_list=glob(os.path.join(base_dir,'*json'))

    leftcut=len(base_dir)

    categories={}
    res = defaultdict(list)

    json_paths=[]
    cnt=1
    for folder in json_folder_list:
        # 카테고리 저장
        categories[folder[leftcut:-5]]=cnt
        cnt+=1

        # json path 저장
        json_paths+=glob(os.path.join(folder,'*json'))


    # 8:2로 학습/검증 데이터 분리
    train_paths=list()
    val_paths=list()

    cnt=0
    for path in json_paths:
        if cnt%5==0:
            val_paths.append(path)
        else:
            train_paths.append(path)
        cnt+=1
    

    
    categories2eng={}
    path_list=[train_paths,val_paths]
    pth_num=0
    for pth in path_list:
        n_id=0

        for json_path in tqdm(pth):
            p=json_path.split('/')
            img_path=os.path.join(p[0],p[1],p[2][:-5],p[3][:-5]+'.jpg')
            img= Image.open(img_path)
            img_width, img_height= img.size

            with open(json_path, 'r') as f:
                temps = json.load(f)

            image_id = temps[0]['Code Name'][:-4]
            res['images'].append({
                    'id':image_id,
                    'width': img_width,
                    'height': img_height,
                    'file_name': temps[0]['Code Name']
                })

            
            if categories[p[2][:-5]] not in categories2eng.keys():
                categories2eng[categories[p[2][:-5]]]=temps[0]['Name']

            for tmp in temps:
            
                mid_x,mid_y=map(float,tmp["Point(x,y)"].split(','))
                x1 = (mid_x-float(tmp['W'])/2)*img_width
                y1 = (mid_y-float(tmp['H'])/2)*img_height
                w  = float(tmp['W'])+img_width
                h  = float(tmp['W'])+img_height
            
                res['annotations'].append({
                    'id': n_id,
                    'image_id':image_id,
                    'category_id': categories[p[2][:-5]],
                    'area': w * h,
                    'bbox': [x1, y1, w, h],
                    'iscrowd': 0
                })
                n_id += 1
        for _, id in categories.items():
            res['categories'].append({
                'id': id,
                'name': categories2eng[id],
            })
        if pth_num==0:
            with open(save_dir+'tarin_annotation.json', 'w') as f:
                json.dump(res, f)
        else:
            with open(save_dir+'val_annotation.json', 'w') as f:
                json.dump(res, f)
        pth_num+=1

In [91]:
convert_to_COCO(base_dir)

  0%|          | 0/800 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]